In [4]:
import random
import torch
from scipy import spatial
import numpy as np

In [8]:
def set_global_seed(seed: int) -> None:
    """
    Sets the seed value for random number generators.

    Parameter
    ---------
    seed: int
        The seed value to use.
    """
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

set_global_seed(1234)

In [9]:
a = torch.randn(2, 2)
b = torch.randn(3, 2) # different row number, for the fun

# Given that cos_sim(u, v) = dot(u, v) / (norm(u) * norm(v))
#                          = dot(u / norm(u), v / norm(v))
# We fist normalize the rows, before computing their dot products via transposition:
a_norm = a / a.norm(dim=1)[:, None]
b_norm = b / b.norm(dim=1)[:, None]
res = torch.mm(a_norm, b_norm.transpose(0,1))
print(res)
#  0.9978 -0.9986 -0.9985
# -0.8629  0.9172  0.9172

tensor([[ 0.5432,  0.9785,  0.9723],
        [ 0.8883, -0.1104,  0.3268]])


In [10]:
# -------
# Let's verify with numpy/scipy if our computations are correct:
a_n = a.numpy()
b_n = b.numpy()
res_n = np.zeros((2, 3))
for i in range(2):
    for j in range(3):
        # cos_sim(u, v) = 1 - cos_dist(u, v)
        res_n[i, j] = 1 - spatial.distance.cosine(a_n[i], b_n[j])
print(res_n)
# [[ 0.9978022  -0.99855876 -0.99854881]
#  [-0.86285472  0.91716063  0.9172349 ]]

[[ 0.54315244  0.97854183  0.97227371]
 [ 0.88826146 -0.11037233  0.32684979]]


In [11]:
normalized_a = torch.nn.functional.normalize(a, dim=1, p=2)
normalized_b = torch.nn.functional.normalize(b, dim=1, p=2)
normalized_a.shape, normalized_b.shape

(torch.Size([2, 2]), torch.Size([3, 2]))

In [21]:
sim_matrix = torch.matmul(normalized_a, normalized_b.T)
print(sim_matrix)
dist_matrix = 1.0 - dist_matrix
print(dist_matrix)

tensor([[ 0.5432,  0.9785,  0.9723],
        [ 0.8883, -0.1104,  0.3268]])
tensor([[0.4568, 0.0215, 0.0277],
        [0.1117, 1.1104, 0.6732]])


In [18]:
sim_matrix = torch.matmul(normalized_a, normalized_b.transpose(0,1))
dist_matrix= 1.0 - sim_matrix
dist_matrix

tensor([[0.4568, 0.0215, 0.0277],
        [0.1117, 1.1104, 0.6732]])